In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.graph_objs as go

class HeatmapPlotter(object):
    def __init__(self, mode='inline', colorscale='Viridis', plotscale=15):
        self.mode = mode
        self.image = None
        self.terminate = False
        self.params = {
            'colorscale': colorscale,
            'plotscale': plotscale
        }
        self.heatmap = {
            'data': [go.Heatmap(
                z = self.image,
                showscale = True,
                colorscale = colorscale,
                colorbar = {
                    'tickformat': '<-d',
                    'tickmode': 'array',
                    'tickvals': []
                }
            )]
        }
        self.app = JupyterDash()
        self.app.layout = html.Div([
            dcc.Graph(
                id = 'graph',
                figure = self.heatmap,
                config={'displayModeBar': False}
            ),
            dcc.Store(
                id = 'image-store',
                data = self.image
            ),
            dcc.Store(
                id = 'params-store',
                data = self.params
            ),
            dcc.Interval(
                id = 'update-store',
                interval = 100,
                n_intervals = 0
            )
        ])
        self.app.clientside_callback(
            """
            function(heat, params) {
              if (!heat) {
                return {
                  layout: {
                    xaxis: {visible: false},
                    yaxis: {visible: false},
                    annotations: [{
                      text: 'No matching data found',
                      xref: 'paper',
                      yref: 'paper',
                      showarrow: false,
                      font: {size: 28}
                    }]
                  }
                };
              }
              const l = 0;
              const r = 110;
              const t = 0;
              const b = 0;
              const numRows = heat.length;
              const numCols = heat[0].length;
              const width = numCols * params.plotscale + r;
              const height = numRows * params.plotscale;
              const minRow = heat.map((row) => {
                return Math.min.apply(Math, row);
              });
              const minZ = Math.min.apply(null, minRow);
              const maxRow = heat.map((row) => {
                return Math.max.apply(Math, row);
              });
              const maxZ = Math.max.apply(null, maxRow);
              return {
                data: [{
                  z: heat,
                  type: 'heatmap',
                  showscale: true,
                  colorscale: params.colorscale,
                  colorbar: {
                    tickformat: '<-d',
                    tickmode: 'array',
                    tickvals: [minZ, maxZ]
                  },
                  zmin: minZ,
                  zmax: maxZ
                }],
                layout: {
                  width,
                  height,
                  margin: {l, r, t, b},
                  xaxis: {
                    showticklabels: false
                  },
                  yaxis: {
                    showticklabels: false
                  }
                }
              };
            }
            """,
            Output('graph', 'figure'),
            Input('image-store', 'data'),
            Input('params-store', 'data')
        )
        self.app.callback(
            [Output('image-store', 'data'),
            Output('update-store', 'disabled')],
            Input('update-store', 'n_intervals')
            )(self.__update_image_store_data)

    def __update_image_store_data(self, n_intervals):
        return [self.image, self.terminate]

    def run_plot(self):
        self.app.run_server(mode = self.mode, host = 'localhost', port = 8050)

    def update_plot(self, image):
        self.image = image

    def stop_plot(self):
        self.terminate = True

